##Creating Positive and negative pairs for cross encoders. It contains both (a,b) and (b,a)

Structure: Title+Text+Body

In [ ]:
import pandas as pd
import json
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/sbert_supervised/android_l_nosw.json','r') as a:
    q = pd.read_json(a,orient='table')

In [ ]:
q.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 8797 entries, 31.0 to 218179.0
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AcceptedAnswerId  3528 non-null   float64
 1   AnswerCount       8797 non-null   float64
 2   AnswerIds         3053 non-null   object 
 3   Attachments       897 non-null    object 
 4   OS                6642 non-null   object 
 5   PostTypeId        8797 non-null   float64
 6   Related           3053 non-null   object 
 7   Score             8797 non-null   float64
 8   Tags              8797 non-null   object 
 9   Text              8797 non-null   object 
 10  Title             8797 non-null   object 
 11  Linked            8797 non-null   object 
dtypes: float64(4), object(8)
memory usage: 893.4+ KB


In [ ]:
data = list()
score = list()
pairs = list()
for i in q.index.values:
    t1 = q.loc[i,'Title'] + q.loc[i,'Tags'] + q.loc[i,'Text']
    for j in list(q.loc[i,'Linked']):
        t2 = q.loc[j,'Title'] + q.loc[j,'Tags'] + q.loc[j,'Text']
        data.append([t1,t2])
        score.append(1)
        pairs.append([i,j])

print(len(data))
print(len(score))

15244
15244


In [ ]:
p_neg = list()
for i in range(0,15224):
    id1 = random.choice(q.index.values)
    id2 = random.choice(q.index.values)
    while([id1,id2] in pairs or [id1,id2] in p_neg or id1==id2):
        id1 = random.choice(q.index.values)
        id2 = random.choice(q.index.values)
    t1 = q.loc[id1,'Title'] + q.loc[id1,'Tags'] + q.loc[id1,'Text']
    t2 = q.loc[id2,'Title'] + q.loc[id2,'Tags'] + q.loc[id2,'Text']
    data.append([t1,t2])
    score.append(0)
    p_neg.append([id1,id2])
print(len(data))
print(len(score))

30468
30468


In [ ]:
import pickle
with open('/content/drive/MyDrive/Linked/data_android_l.txt','wb') as a:
    pickle.dump(data,a)
    pickle.dump(score,a)

#Modeling Cross Encoder

In [1]:
import pandas as pd
import json
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Linked/data_android_l.txt','rb') as a:
    data = pickle.load(a)
    score = pickle.load(a)
print(data[0][0])
print(type(score[0]))

##Training Cross Encoder

In [2]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.5MB 7.6MB/s 
     |████████████████████████████████| 1.2MB 49.8MB/s 
     |████████████████████████████████| 2.9MB 53.7MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=24140979762cfc42046ff06c26934c6dd0477b4ab30960f377a6a2517672f445
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f447d7b11a77e7a61647ccd3251f358a672e40c344b191b1cc8f8262837c455c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [3]:
from sentence_transformers import models, losses, util, SentenceTransformer, SentencesDataset, InputExample, evaluation
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator as ceb

from torch import nn
from torch.utils.data import DataLoader
import torch

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [7]:
data,score = shuffle(data,score)
train_data,dev_data,train_score,dev_score = train_test_split(data,score,test_size=0.1,random_state=13)

In [8]:
model = CrossEncoder('bert-base-uncased',num_labels=1)

In [9]:
train_examples = list()
dev_examples = list()
for i in range(0,len(train_data)):
    train_examples.append(InputExample(texts=[train_data[i][0],train_data[i][1]], label=train_score[i]))

for i in range(0,len(dev_data)):
    dev_examples.append(InputExample(texts=[dev_data[i][0],dev_data[i][1]], label=dev_score[i]))

In [28]:
batch_size=8
num_epochs = 6
model_path = '/content/drive/MyDrive/Linked'

In [21]:
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

In [22]:
print(torch.cuda.is_available())

True


In [26]:
evaluator_dev = ceb.from_input_examples(dev_examples,name='android-dev')

In [27]:
import math
warmup_steps = math.ceil(len(train_dataset) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

In [28]:
x = list()
y = list()
def callback_model(score,epoch,steps):
    if(steps==-1):
        steps=0
        epoch+=1
    x.append(float(epoch + steps/3428))
    y.append(float(score))
    print('Score after epoch ', str(epoch),' and steps ', str(steps),'is ',str(score))

In [29]:
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator_dev,
          epochs=num_epochs,
          evaluation_steps=1200,
          warmup_steps = warmup_steps,
          output_path=model_path,
          use_amp = True,
          callback = callback_model
          )

RuntimeError: ignored

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(x,y)

In [ ]:
with open('/content/drive/MyDrive/Linked/Accuracy_score.txt','wb') as f:
    pickle.dump(x,f)
    pickle.dump(y,f)

In [ ]:
plt.savefig('/content/drive/MyDrive/Linked/final_accuarcy.png')

#Labelling android dataset

In [29]:
ce_model = CrossEncoder('bert-base-uncased',max_length=512)
ce_model_m = CrossEncoder(model_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

##Positive Pair Generation 

In [18]:
with open('/content/drive/MyDrive/sbert_supervised/android_nosw.json','r') as f:
    qr = pd.read_json(f,orient='table')

In [19]:
def lts(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += str(ele) + ' '  
    
    # return string   
    return str1  

In [20]:
qr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17132 entries, 31 to 219026
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AcceptedAnswerId  6631 non-null   float64
 1   AnswerCount       17132 non-null  float64
 2   AnswerIds         17132 non-null  object 
 3   Attachments       4246 non-null   object 
 4   OS                4246 non-null   object 
 5   PostTypeId        17132 non-null  int64  
 6   Related           17132 non-null  object 
 7   Score             17132 non-null  int64  
 8   Tags              17132 non-null  object 
 9   Text              17132 non-null  object 
 10  Title             17132 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.6+ MB


In [35]:
data_pos = list()
cnt = 0
num_pairs = 100
related_not_found = 0
al_pos = list()
for i in qr.index.values:
    if(cnt>=num_pairs):
        break
    if(len(qr.loc[i,'Related'])!=0):
        for j in qr.loc[i,'Related']:
            if( ( [j,i] in al_pos or [i,j] in al_pos) != 1 ):
                # try:
                t1 = qr.loc[i,'Title'] + lts(qr.loc[i,'Tags']) + qr.loc[i,'Text']
                t2 = qr.loc[j,'Title'] + lts(qr.loc[j,'Tags']) + qr.loc[j,'Text']
                cnt+=1
                data_pos.append([t1,t2])
                al_pos.append([ i , j ])
                    # break
                # except:
                #     related_not_found+=1
                #     # print('Related not found,parent id: ',i,' id:',j,'overall count: ',related_not_found)
print(related_not_found)
print(len(data_pos))

0
106


In [36]:
acc_bert = ce_model.predict(data_pos)
print(sum(acc_bert)/106)
acc_m = ce_model_m.predict(data_pos)
print(sum(acc_m)/106)

0.5723572485851791
0.8591707203916674


In [37]:
import random
num_pairs = len(data_pos)
data_neg = list()
al_acc = list()
for i in range(0,num_pairs):
    id1 = random.choice(qr.index.values)
    id2 = random.choice(qr.index.values)
    while(id2 in qr.loc[id1,'Related'] or id1 in qr.loc[id2,'Related'] or [id1,id2] in al_acc or [id2,id1] in al_acc):
         id1 = random.choice(qr.index.values)
         id2 = random.choice(qr.index.values)
    t1 = qr.loc[id1,'Title'] + lts(qr.loc[id1,'Tags']) + qr.loc[id1,'Text']
    t2 = qr.loc[id2,'Title'] + lts(qr.loc[id2,'Tags']) + qr.loc[id2,'Text']
    data_neg.append([t1,t2])
    al_acc.append([id1,id2])
print(len(data_neg))

106


In [38]:
acc_bert_n = ce_model.predict(data_neg)
print(sum(acc_bert_n)/106)
acc_m_n = ce_model_m.predict(data_neg)
print(sum(acc_m_n)/106)

0.5840784496856186
0.06935349212859741


In [73]:
print((acc_m_n>0.9))

[False False False False False False  True False False False False  True
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False False False False False False False False  True
 False False False False False False False False  True False  True False
 False False False False False False False False False False False False
 False False False False False False False False False False]


In [76]:
print(data_neg[7][0] + '\n\n' + data_neg[7][1]) 

number pattern matching attempt android 2.3(maxx msd7 ) factory-reset start phone(maxx msd7),i get " number invalid pattern matching attempt " error android 2.3.i try perform factory reset starup . get recovery menu startup press volume - button not get factory reset option . test item test clear flash option . . ? 

create profile disable profile tasker couple profile define tasker like enable travel abroad . tasker not recognize state " travel abroad " , like set state manually disable profile relate meta profile " travel abroad " . envision tasker : category : travel abroad ( switch / ) profile 1 ( switch / ) profile 2 ( switch / ) profile 3 ( switch / ) ... 
